# <center>Implementing LDA in Python</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [4]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import pandas as pd
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
import nltk
from nltk.corpus import stopwords

#vis
import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [11]:
stopwords = stopwords.words("dutch")

In [12]:
print(stopwords)

['de', 'en', 'van', 'ik', 'te', 'dat', 'die', 'in', 'een', 'hij', 'het', 'niet', 'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 'niets', 'uw', 'iemand', 'geweest', 'andere']


# Getting Data

In [34]:

url = '/Users/dj/Python - UvA/DSP/data/' # Insert your path 
numbers = ["birds", 'elephant', 'fish']
ending = '.txt'

texts = []
for i in numbers:
    p = url + i + ending
    f = open(p,'r')
    text = f.read()
    texts.append(text)
    

data = texts

print (data[2][0:90])

﻿Vissen zijn aquatische, craniate, kieuwdragende dieren die geen ledematen met vingers heb


In [36]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("nl_core_news_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

﻿vogel groep warmbloedig gewerveld dier deel uitmaken klasse kenmerken veer tandenloz snav


In [15]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['jarige', 'man', 'veroordelen', 'jaar', 'gevangenisstraf', 'nacht', 'ogenschijnlijk', 'pont', 'mes', 'man', 'insteken', 'daarbij', 'ernstig', 'gewond', 'raken']


In [25]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100) # min_counts = Minimum counts that it is being taken into consideration
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)


In [26]:
print (data_bigrams_trigrams[2])

['diefstal_geweld', 'vereniging']


In [17]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


In [18]:
# id2word = corpora.Dictionary(all_texts)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# print (corpus[0][0:20])

# word = id2word[[0][:1][0]]
# print (word)

In [19]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


In [20]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.1327212), (1, 0.15550919), (2, 0.26527536), (3, 0.37257102), (4, 0.07392319)]
[(3, 0.37257102), (2, 0.26527536), (1, 0.15550919), (0, 0.1327212), (4, 0.07392319)]


In [49]:
# lda_model.save("/Users/dj/Documents/GitHub/DSP/topic_modeling_textbook-main/models/test_model.model")

In [50]:
# new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [21]:
test_doc = corpus[-1]

vector = lda_model[test_doc] # lda_model can be replaced by new_model
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.13272117), (1, 0.15550913), (2, 0.26527548), (3, 0.3725708), (4, 0.07392344)]
[(3, 0.3725708), (2, 0.26527548), (1, 0.15550913), (0, 0.13272117), (4, 0.07392344)]


## Vizualizing the Data

In [22]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.373125 -0.025084       1        1  29.733767
1      0.104710  0.304016       2        1  24.152732
2     -0.276770 -0.178179       3        1  16.698878
4     -0.264489  0.211956       4        1  16.517875
0      0.063424 -0.312709       5        1  12.896747, topic_info=           Term         Freq        Total Category  logprob  loglift
33         niet  1933.000000  1933.000000  Default  10.0000  10.0000
22      artikel   802.000000   802.000000  Default   9.0000   9.0000
82       beroep   753.000000   753.000000  Default   8.0000   8.0000
214   verdenken   475.000000   475.000000  Default   7.0000   7.0000
435   appellant   487.000000   487.000000  Default   6.0000   6.0000
...         ...          ...          ...      ...      ...      ...
695     voldoen   100.073780   100.747521   Topic5  -4.5589   2.0415
581          nu   101.163937   101.846705   Topic5  -4.5481   2.0415
2018   bijstand    95.837269    96.517696   Topic5  -4.6022   2.0411
1271      dagen    81.642173    82.308335   Topic5  -4.7625   2.0401
112    afwijzen    86.521448   397.578704   Topic5  -4.7045   0.5232

[72 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
326       4  0.996606   aanleiding
434       3  0.994975  aannemelijk
654       5  0.997658     aanvraag
112       2  0.782235     afwijzen
112       5  0.218825     afwijzen
...     ...       ...          ...
43        2  0.955890       woning
43        4  0.039663       woning
44        3  0.974313         zaak
44        4  0.023534         zaak
652       1  0.999328         zijn

[69 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 5, 1])

In [ ]:
# Accuracy metric
# Grid search 